# Test Funnel of Verification (FoVe) Functions

This notebook tests the Funnel of Verification functions across all three providers:
1. `funnel_of_verification_anthropic` - Claude with web_search tool
2. `funnel_of_verification_google` - Gemini with grounded search
3. `funnel_of_verification_perplexity` - Perplexity with web search

## FoVe Pipeline Overview
- **Step 1**: Gather broad information about the item (web search)
- **Step 2**: Extract concise answer from context (no web search) - can trigger early exit
- **Step 3**: Skeptically verify the answer (web search)
- **Step 4**: Format output as strict JSON (no web search)

## Test Goal
Test with **1 search question** and **5 company inputs** to verify structured DataFrame output with columns: `search_input`, `answer`, `url`, `confidence`, `multiple_entities`

In [ ]:
import sys
import os
import json
import pandas as pd
from tqdm import tqdm
import time

# Use local src
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, src_path)

from llm_web_research.calls import (
    funnel_of_verification_anthropic,
    funnel_of_verification_google,
    funnel_of_verification_perplexity
)

print("Funnel of Verification functions loaded successfully")

In [ ]:
# Load API keys from .env file
from dotenv import load_dotenv, find_dotenv

os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
_ = load_dotenv(find_dotenv())

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")

# Change back to llm-web-research directory
os.chdir('/Users/chrissoria/Documents/Research/llm-web-research')

# Verify keys loaded
print("API keys loaded:")
print(f"  Anthropic: {'Y' if anthropic_api_key else 'N'}")
print(f"  Google: {'Y' if google_api_key else 'N'}")
print(f"  Perplexity: {'Y' if perplexity_api_key else 'N'}")

In [ ]:
# Test parameters: 1 question, 5 companies
search_question = "current CEO"
answer_format = "name"
search_inputs = ["Microsoft", "Apple Inc", "Amazon", "Google", "Meta"]

# Common parameters
additional_instructions = ""
creativity = 0
time_delay = 3  # seconds between requests

print(f"Search question: {search_question}")
print(f"Answer format: {answer_format}")
print(f"\nCompanies to search ({len(search_inputs)}):")
for i, item in enumerate(search_inputs, 1):
    print(f"  {i}. {item}")

---
## Test 1: Anthropic (Claude with web_search tool)

Uses `tool_choice` with `input_schema` to force JSON output.

In [ ]:
if anthropic_api_key:
    import anthropic
    
    print("Testing Funnel of Verification with Anthropic...\n")
    client = anthropic.Anthropic(api_key=anthropic_api_key)
    user_model = "claude-sonnet-4-20250514"
    
    results = []
    
    for idx, item in enumerate(tqdm(search_inputs, desc="Processing")):
        if idx > 0:
            time.sleep(time_delay)
        
        print(f"\n{'#'*80}")
        print(f"# PROCESSING: {item}")
        print(f"{'#'*80}")
        
        try:
            result = funnel_of_verification_anthropic(
                item=item,
                search_question=search_question,
                answer_format=answer_format,
                additional_instructions=additional_instructions,
                client=client,
                user_model=user_model,
                creativity=creativity,
                verbose=True  # Enable verbose output
            )
            
            # Parse JSON result
            parsed = json.loads(result)
            results.append({
                'search_input': item,
                'answer': parsed.get('answer', ''),
                'url': parsed.get('url', ''),
                'confidence': parsed.get('confidence', ''),
                'multiple_entities': parsed.get('multiple_entities', '0')
            })
            
        except json.JSONDecodeError as e:
            print(f"JSON parse error for {item}: {e}")
            print(f"Raw result: {result}")
            results.append({
                'search_input': item,
                'answer': f'JSON Error: {result[:100]}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
        except Exception as e:
            print(f"Error for {item}: {e}")
            results.append({
                'search_input': item,
                'answer': f'Error: {e}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
    
    # Create DataFrame
    df_anthropic = pd.DataFrame(results)
    
    print("\n" + "="*60)
    print("ANTHROPIC RESULTS")
    print("="*60)
    print(f"\nDataFrame shape: {df_anthropic.shape}")
    print(f"Columns: {list(df_anthropic.columns)}")
    print("\n")
    display(df_anthropic)
else:
    print("Skipping: Anthropic API key not found")

---
## Test 2: Google (Gemini with grounded search)

Uses `responseMimeType: application/json` to force JSON output.

In [ ]:
if google_api_key:
    import requests
    
    print("Testing Funnel of Verification with Google...\n")
    
    user_model = "gemini-2.5-flash"
    url = f"https://generativelanguage.googleapis.com/v1beta/models/{user_model}:generateContent"
    headers = {
        "x-goog-api-key": google_api_key,
        "Content-Type": "application/json"
    }
    
    def make_google_request(url, headers, payload, max_retries=5):
        """Make Google API request with retry logic."""
        for attempt in range(max_retries):
            response = requests.post(url, headers=headers, json=payload)
            if response.status_code == 429:
                wait_time = 2 ** attempt
                print(f"  Rate limited, waiting {wait_time}s...")
                time.sleep(wait_time)
                continue
            response.raise_for_status()
            return response.json()
        raise Exception("Max retries exceeded")
    
    results = []
    
    for idx, item in enumerate(tqdm(search_inputs, desc="Processing")):
        if idx > 0:
            time.sleep(time_delay)
        
        print(f"\n{'#'*80}")
        print(f"# PROCESSING: {item}")
        print(f"{'#'*80}")
        
        try:
            result = funnel_of_verification_google(
                item=item,
                search_question=search_question,
                answer_format=answer_format,
                additional_instructions=additional_instructions,
                url=url,
                headers=headers,
                creativity=creativity,
                make_google_request=make_google_request,
                verbose=True  # Enable verbose output
            )
            
            # Parse JSON result
            parsed = json.loads(result)
            results.append({
                'search_input': item,
                'answer': parsed.get('answer', ''),
                'url': parsed.get('url', ''),
                'confidence': parsed.get('confidence', ''),
                'multiple_entities': parsed.get('multiple_entities', '0')
            })
            
        except json.JSONDecodeError as e:
            print(f"JSON parse error for {item}: {e}")
            print(f"Raw result: {result}")
            results.append({
                'search_input': item,
                'answer': f'JSON Error: {result[:100]}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
        except Exception as e:
            print(f"Error for {item}: {e}")
            results.append({
                'search_input': item,
                'answer': f'Error: {e}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
    
    # Create DataFrame
    df_google = pd.DataFrame(results)
    
    print("\n" + "="*60)
    print("GOOGLE RESULTS")
    print("="*60)
    print(f"\nDataFrame shape: {df_google.shape}")
    print(f"Columns: {list(df_google.columns)}")
    print("\n")
    display(df_google)
else:
    print("Skipping: Google API key not found")

---
## Test 3: Perplexity

Uses `response_format` with `json_schema` to force JSON output.

In [ ]:
if perplexity_api_key:
    from perplexity import Perplexity
    
    print("Testing Funnel of Verification with Perplexity...\n")
    client = Perplexity(api_key=perplexity_api_key)
    user_model = "sonar"
    
    results = []
    
    for idx, item in enumerate(tqdm(search_inputs, desc="Processing")):
        if idx > 0:
            time.sleep(time_delay)
        
        print(f"\n{'#'*80}")
        print(f"# PROCESSING: {item}")
        print(f"{'#'*80}")
        
        try:
            result = funnel_of_verification_perplexity(
                item=item,
                search_question=search_question,
                answer_format=answer_format,
                additional_instructions=additional_instructions,
                client=client,
                user_model=user_model,
                creativity=creativity,
                verbose=True  # Enable verbose output
            )
            
            # Parse JSON result
            parsed = json.loads(result)
            results.append({
                'search_input': item,
                'answer': parsed.get('answer', ''),
                'url': parsed.get('url', ''),
                'confidence': parsed.get('confidence', ''),
                'multiple_entities': parsed.get('multiple_entities', '0')
            })
            
        except json.JSONDecodeError as e:
            print(f"JSON parse error for {item}: {e}")
            print(f"Raw result: {result}")
            results.append({
                'search_input': item,
                'answer': f'JSON Error: {result[:100]}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
        except Exception as e:
            print(f"Error for {item}: {e}")
            results.append({
                'search_input': item,
                'answer': f'Error: {e}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
    
    # Create DataFrame
    df_perplexity = pd.DataFrame(results)
    
    print("\n" + "="*60)
    print("PERPLEXITY RESULTS")
    print("="*60)
    print(f"\nDataFrame shape: {df_perplexity.shape}")
    print(f"Columns: {list(df_perplexity.columns)}")
    print("\n")
    display(df_perplexity)
else:
    print("Skipping: Perplexity API key not found")

---
## Results Comparison

Side-by-side comparison of all three providers.

In [ ]:
# Build comparison DataFrame
comparison_data = []

for item in search_inputs:
    row = {'company': item}
    
    # Anthropic
    if 'df_anthropic' in dir():
        match = df_anthropic[df_anthropic['search_input'] == item]
        if not match.empty:
            row['anthropic_answer'] = match.iloc[0]['answer']
            row['anthropic_conf'] = match.iloc[0]['confidence']
    
    # Google
    if 'df_google' in dir():
        match = df_google[df_google['search_input'] == item]
        if not match.empty:
            row['google_answer'] = match.iloc[0]['answer']
            row['google_conf'] = match.iloc[0]['confidence']
    
    # Perplexity
    if 'df_perplexity' in dir():
        match = df_perplexity[df_perplexity['search_input'] == item]
        if not match.empty:
            row['perplexity_answer'] = match.iloc[0]['answer']
            row['perplexity_conf'] = match.iloc[0]['confidence']
    
    comparison_data.append(row)

df_comparison = pd.DataFrame(comparison_data)

print("="*80)
print("COMPARISON: CEO Answers Across Providers")
print("="*80)
print(f"\nQuestion: '{search_question}'")
print(f"Format: '{answer_format}'\n")
display(df_comparison)

In [ ]:
# Check agreement between providers
print("\nAgreement Analysis:")
print("-" * 40)

if 'df_comparison' in dir() and len(df_comparison) > 0:
    answer_cols = [c for c in df_comparison.columns if '_answer' in c]
    
    if len(answer_cols) >= 2:
        for idx, row in df_comparison.iterrows():
            answers = [row.get(c, 'N/A') for c in answer_cols if pd.notna(row.get(c))]
            unique_answers = set(a.lower().strip() for a in answers if a and a != 'N/A')
            
            if len(unique_answers) == 1:
                status = "All agree"
            elif len(unique_answers) == 0:
                status = "No answers"
            else:
                status = "DISAGREE"
            
            print(f"{row['company']}: {status}")
            if status == "DISAGREE":
                for col in answer_cols:
                    provider = col.replace('_answer', '')
                    print(f"  - {provider}: {row.get(col, 'N/A')}")
    else:
        print("Need at least 2 providers to compare.")
else:
    print("No comparison data available.")

---
## Test 4: Edge Cases (Ambiguous & Unknown Queries)

Test cases designed to trigger early exits:
- **Ambiguous names**: "John Smith" - too many people with this name, should trigger RESPONSE NOT CONFIDENT
- **Unknown entities**: "XYZ Fake Company 12345" - should trigger ANSWER NOT FOUND
- **Common name with specific question**: "Michael Johnson height" - many athletes/people named this

**Expected behavior**: confidence = 0 for all ambiguous cases

In [ ]:
# Edge case test parameters
edge_cases = [
    {"item": "John Smith", "question": "height", "format": "feet and inches"},
    {"item": "Michael Johnson", "question": "height", "format": "feet and inches"},
    {"item": "XYZ Fake Company 12345", "question": "CEO", "format": "name"},
    {"item": "David Williams", "question": "net worth", "format": "USD amount"},
    {"item": "Apple", "question": "founder", "format": "name"},  # Fruit or company?
]

print("Edge Case Test Items:")
print("="*60)
for i, tc in enumerate(edge_cases, 1):
    print(f"{i}. '{tc['item']}' - {tc['question']} (format: {tc['format']})")
print("\nExpected: Most should return confidence=0 due to ambiguity")

In [ ]:
if anthropic_api_key:
    import anthropic
    
    print("Testing Edge Cases with Anthropic...\n")
    client = anthropic.Anthropic(api_key=anthropic_api_key)
    user_model = "claude-sonnet-4-20250514"
    
    edge_results = []
    
    for idx, tc in enumerate(tqdm(edge_cases, desc="Processing edge cases")):
        if idx > 0:
            time.sleep(time_delay)
        
        print(f"\n{'#'*80}")
        print(f"# EDGE CASE: {tc['item']} - {tc['question']}")
        print(f"{'#'*80}")
        
        try:
            result = funnel_of_verification_anthropic(
                item=tc['item'],
                search_question=tc['question'],
                answer_format=tc['format'],
                additional_instructions=additional_instructions,
                client=client,
                user_model=user_model,
                creativity=creativity,
                verbose=True  # Enable verbose output
            )
            
            parsed = json.loads(result)
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': parsed.get('answer', ''),
                'url': parsed.get('url', ''),
                'confidence': parsed.get('confidence', ''),
                'multiple_entities': parsed.get('multiple_entities', '0')
            })
            
            print(f"\n>>> FINAL: Answer: {parsed.get('answer')}, Confidence: {parsed.get('confidence')}, Multiple: {parsed.get('multiple_entities')}")
            
        except json.JSONDecodeError as e:
            print(f"  JSON Error: {e}")
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': f'JSON Error',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
        except Exception as e:
            print(f"  Error: {e}")
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': f'Error: {e}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
    
    df_edge_anthropic = pd.DataFrame(edge_results)
    
    print("\n" + "="*60)
    print("ANTHROPIC EDGE CASE RESULTS")
    print("="*60)
    display(df_edge_anthropic)
    
    # Check confidence distribution
    print("\nConfidence Distribution:")
    print(df_edge_anthropic['confidence'].value_counts())
    print("\nMultiple Entities Distribution:")
    print(df_edge_anthropic['multiple_entities'].value_counts())
else:
    print("Skipping: Anthropic API key not found")

In [ ]:
if google_api_key:
    import requests
    
    print("Testing Edge Cases with Google...\n")
    
    user_model = "gemini-2.5-flash"
    url = f"https://generativelanguage.googleapis.com/v1beta/models/{user_model}:generateContent"
    headers = {
        "x-goog-api-key": google_api_key,
        "Content-Type": "application/json"
    }
    
    def make_google_request(url, headers, payload, max_retries=5):
        for attempt in range(max_retries):
            response = requests.post(url, headers=headers, json=payload)
            if response.status_code == 429:
                wait_time = 2 ** attempt
                print(f"  Rate limited, waiting {wait_time}s...")
                time.sleep(wait_time)
                continue
            response.raise_for_status()
            return response.json()
        raise Exception("Max retries exceeded")
    
    edge_results = []
    
    for idx, tc in enumerate(tqdm(edge_cases, desc="Processing edge cases")):
        if idx > 0:
            time.sleep(time_delay)
        
        print(f"\n{'#'*80}")
        print(f"# EDGE CASE: {tc['item']} - {tc['question']}")
        print(f"{'#'*80}")
        
        try:
            result = funnel_of_verification_google(
                item=tc['item'],
                search_question=tc['question'],
                answer_format=tc['format'],
                additional_instructions=additional_instructions,
                url=url,
                headers=headers,
                creativity=creativity,
                make_google_request=make_google_request,
                verbose=True  # Enable verbose output
            )
            
            parsed = json.loads(result)
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': parsed.get('answer', ''),
                'url': parsed.get('url', ''),
                'confidence': parsed.get('confidence', ''),
                'multiple_entities': parsed.get('multiple_entities', '0')
            })
            
            print(f"\n>>> FINAL: Answer: {parsed.get('answer')}, Confidence: {parsed.get('confidence')}, Multiple: {parsed.get('multiple_entities')}")
            
        except json.JSONDecodeError as e:
            print(f"  JSON Error: {e}")
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': f'JSON Error',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
        except Exception as e:
            print(f"  Error: {e}")
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': f'Error: {e}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
    
    df_edge_google = pd.DataFrame(edge_results)
    
    print("\n" + "="*60)
    print("GOOGLE EDGE CASE RESULTS")
    print("="*60)
    display(df_edge_google)
    
    print("\nConfidence Distribution:")
    print(df_edge_google['confidence'].value_counts())
    print("\nMultiple Entities Distribution:")
    print(df_edge_google['multiple_entities'].value_counts())
else:
    print("Skipping: Google API key not found")

In [ ]:
if perplexity_api_key:
    from perplexity import Perplexity
    
    print("Testing Edge Cases with Perplexity...\n")
    client = Perplexity(api_key=perplexity_api_key)
    user_model = "sonar"
    
    edge_results = []
    
    for idx, tc in enumerate(tqdm(edge_cases, desc="Processing edge cases")):
        if idx > 0:
            time.sleep(time_delay)
        
        print(f"\n{'#'*80}")
        print(f"# EDGE CASE: {tc['item']} - {tc['question']}")
        print(f"{'#'*80}")
        
        try:
            result = funnel_of_verification_perplexity(
                item=tc['item'],
                search_question=tc['question'],
                answer_format=tc['format'],
                additional_instructions=additional_instructions,
                client=client,
                user_model=user_model,
                creativity=creativity,
                verbose=True  # Enable verbose output
            )
            
            parsed = json.loads(result)
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': parsed.get('answer', ''),
                'url': parsed.get('url', ''),
                'confidence': parsed.get('confidence', ''),
                'multiple_entities': parsed.get('multiple_entities', '0')
            })
            
            print(f"\n>>> FINAL: Answer: {parsed.get('answer')}, Confidence: {parsed.get('confidence')}, Multiple: {parsed.get('multiple_entities')}")
            
        except json.JSONDecodeError as e:
            print(f"  JSON Error: {e}")
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': f'JSON Error',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
        except Exception as e:
            print(f"  Error: {e}")
            edge_results.append({
                'search_input': tc['item'],
                'question': tc['question'],
                'answer': f'Error: {e}',
                'url': '',
                'confidence': '0',
                'multiple_entities': '0'
            })
    
    df_edge_perplexity = pd.DataFrame(edge_results)
    
    print("\n" + "="*60)
    print("PERPLEXITY EDGE CASE RESULTS")
    print("="*60)
    display(df_edge_perplexity)
    
    print("\nConfidence Distribution:")
    print(df_edge_perplexity['confidence'].value_counts())
    print("\nMultiple Entities Distribution:")
    print(df_edge_perplexity['multiple_entities'].value_counts())
else:
    print("Skipping: Perplexity API key not found")

In [ ]:
# Compare edge case results across providers
print("="*80)
print("EDGE CASE COMPARISON: Ambiguous Query Handling")
print("="*80)

edge_comparison = []
for tc in edge_cases:
    row = {'item': tc['item'], 'question': tc['question']}
    
    if 'df_edge_anthropic' in dir():
        match = df_edge_anthropic[df_edge_anthropic['search_input'] == tc['item']]
        if not match.empty:
            row['anthropic_answer'] = match.iloc[0]['answer'][:30] + '...' if len(str(match.iloc[0]['answer'])) > 30 else match.iloc[0]['answer']
            row['anthropic_conf'] = match.iloc[0]['confidence']
    
    if 'df_edge_google' in dir():
        match = df_edge_google[df_edge_google['search_input'] == tc['item']]
        if not match.empty:
            row['google_answer'] = match.iloc[0]['answer'][:30] + '...' if len(str(match.iloc[0]['answer'])) > 30 else match.iloc[0]['answer']
            row['google_conf'] = match.iloc[0]['confidence']
    
    if 'df_edge_perplexity' in dir():
        match = df_edge_perplexity[df_edge_perplexity['search_input'] == tc['item']]
        if not match.empty:
            row['perplexity_answer'] = match.iloc[0]['answer'][:30] + '...' if len(str(match.iloc[0]['answer'])) > 30 else match.iloc[0]['answer']
            row['perplexity_conf'] = match.iloc[0]['confidence']
    
    edge_comparison.append(row)

df_edge_comparison = pd.DataFrame(edge_comparison)
display(df_edge_comparison)

# Summary
print("\n" + "-"*40)
print("SUMMARY: How well did models handle ambiguity?")
print("-"*40)
conf_cols = [c for c in df_edge_comparison.columns if '_conf' in c]
for col in conf_cols:
    provider = col.replace('_conf', '')
    if col in df_edge_comparison.columns:
        zeros = (df_edge_comparison[col] == '0').sum()
        total = df_edge_comparison[col].notna().sum()
        print(f"{provider}: {zeros}/{total} returned confidence=0 (expected for ambiguous queries)")

---
## Summary

**Test 1-3: Standard CEO Queries**
- Search question: "current CEO"
- Answer format: "name"
- Companies: Microsoft, Apple Inc, Amazon, Google, Meta
- Expected: High confidence (1) for all

**Test 4: Edge Cases (Ambiguous Queries)**
- "John Smith" height → Should return confidence=0 (too many people)
- "Michael Johnson" height → Should return confidence=0 (common name)
- "XYZ Fake Company" CEO → Should return confidence=0 (doesn't exist)
- "David Williams" net worth → Should return confidence=0 (too many people)
- "Apple" founder → May be ambiguous (fruit vs company)

**Output Format:**
```
| search_input | answer         | url                    | confidence |
|--------------|----------------|------------------------|------------|
| Microsoft    | Satya Nadella  | https://microsoft.com  | 1          |
| John Smith   | Information... | (empty)                | 0          |
```

**Providers Tested:**
1. Anthropic - `tool_choice` JSON forcing
2. Google - `responseMimeType` JSON forcing  
3. Perplexity - `json_schema` JSON forcing